In [5]:
import numpy as np
import pandas as pd
from osgeo import gdal
from gdalconst import *
#import osr
import matplotlib.pyplot as plt
%matplotlib qt

#Import bokeh 
from bokeh.plotting import *
from bokeh.models import ColumnDataSource, Label
from bokeh.layouts import layout

#package in this working directory
from utils.colormap import RGBAColorMapper
import netCDF4
from bokeh.palettes import RdBu11
#import color palettes for plotting raster
from bokeh.palettes import *




/Users/bmelosh/anaconda2/envs/geospatial/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [10]:
#Read a raster
Test_data = "/Users/bmelosh/VagrantBoxes/LSDTopoTools/Topographic_projects/Test_data/"
Test = gdal.Open(Test_data + "/Mandakini.bil")#Get raster data
M_DEM = Test.ReadAsArray()

#Read stream data
name = "Mandakini_fullProfileMC_forced_0.45_3_1258909000_10_80_281_for_Arc.csv" #Set file name
path = "/Users/bmelosh/VagrantBoxes/LSDTopoTools/Topographic_projects/Test_data/" #Set path to data
#for csv files
data = np.genfromtxt((path+name), delimiter=',', skip_header=1, names=['id', 'x', 'y', 'chan_number', 'reciever_chan','node_on_reciever_chan', 'node', 'row','column', 'flow_distance', 'chi', 'elevation', 'drainage_area', 'n_data_points', 'm_mean','m_st_dev', 'm_std_err', 'b_mean', 'b_st_dev', 'b_std_err', 'DW_mean', 'DW_st_dev', 'DW_std_err', 'fitted_elev_mean', 'fitted_elev_stdev', 'fitted_elev_std_err'])
#for tree files
#data = np.genfromtxt((path+name), delimiter=' ', skip_header=1, names=['chan_number', 'reciever_chan','node_on_reciever_chan', 'node', 'row','column', 'flow_distance', 'chi', 'elevation', 'drainage_area', 'n_data_points', 'm_mean','m_st_dev', 'm_std_err', 'b_mean', 'b_st_dev', 'b_std_err', 'DW_mean', 'DW_st_dev', 'DW_std_err', 'fitted_elev_mean', 'fitted_elev_stdev', 'fitted_elev_std_err'])


In [3]:



def plot_coord(Test):

    (upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = Test.GetGeoTransform()
    dx = x_size
    dy = y_size
    nx, ny = Test.RasterXSize, Test.RasterYSize  # Size of the original raster

    xllcenter = upper_left_x + dx/2  # x coordinate center of lower left pxl
    yllcenter = upper_left_y - dx/2 # y coordinate center of lower left pxl   - (ny-1)*dx 

    #Create arrays of the x and y coordinates of each pixel (the axes)
    xcoordinates = [x*dx + xllcenter for x in range(nx)]
    ycoordinates = [y*dy + yllcenter for y in range(ny)]

    #Create 2 2d grids describing x and y coordinates
    X,Y = np.meshgrid(xcoordinates, ycoordinates) 


    fig = plt.contourf(X, Y, M_DEM, levels=np.linspace(np.amin(M_DEM[M_DEM > 0]),np.amax(M_DEM), 50))

    return X, Y


In [63]:
(upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = Test.GetGeoTransform()
dx = x_size
dy = y_size
nx, ny = Test.RasterXSize, Test.RasterYSize  # Size of the original raster

xllcenter = upper_left_x + dx/2  # x coordinate center of lower left pxl
yllcenter = upper_left_y - dx/2 # y coordinate center of lower left pxl   - (ny-1)*dx 

#Create arrays of the x and y coordinates of each pixel (the axes)
xcoordinates = [x*dx + xllcenter for x in range(nx)]
ycoordinates = [y*dy + yllcenter for y in range(ny)]

#Create 2 2d grids describing x and y coordinates
X,Y = np.meshgrid(xcoordinates, ycoordinates) 


fig = plt.contourf(X, Y, M_DEM, levels=np.linspace(np.amin(M_DEM[M_DEM > 0]),np.amax(M_DEM), 50))
    

In [10]:
[X, Y] = plot_coord(Test)

In [180]:
cm = plt.cm.gist_earth


fig, ax = plt.subplots(3, figsize=(10,10), sharex=False) #define 2 subfigures

ax[0].contourf(X, Y, M_DEM, levels=np.linspace(np.amin(M_DEM[M_DEM > 0]),np.amax(M_DEM), 50))
ax[0].plot(data['x'], data['y'], 'ro',  markersize=2)
#ax[0].imshow(fA, interpolation='nearest', vmin=0, cmap=plt.cm.gray) 
ax[0].set_aspect('equal')
ax[0].set_title('Terrain map with stream channel')
ax[0].set_ylabel('Northing')
ax[0].set_xlabel('Easting')


ax[1].scatter(data['flow_distance'], data['elevation'], c=data['elevation'], cmap=cm)
ax[1].set_title('Stream Map')
ax[1].set_ylabel('Elevation (m)')
ax[1].set_xlabel('Distance (m)')


ax[2].scatter(data['chi'], data['elevation'], c=data['elevation'], cmap=plt.get_cmap("jet"))
ax[2].text(3, 2500, 'm/n = 0.45')
ax[2].set_ylabel('Elevation (m)')
ax[2].set_xlabel('Chi (X)')


plt.show()

In [18]:
img = (X, Y)


In [181]:

#My bokeh figure
mn = " (m/n = 0.45)"

# output to static HTML file
output_file("linked_brushing.html")

# NEW: create a column data source for the plots to share
stream_source = ColumnDataSource(data=dict(x0=data['flow_distance'], x1=data['chi'], y=data['elevation']))

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=350, height=350, title='Stream Profile')
left.circle('x0', 'y', source=stream_source)
left.xaxis.axis_label = 'Flow Distance (m)'
left.yaxis.axis_label = 'Elevation (m)'

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=350, height=350, title='Chi Profile'+ mn)
right.circle('x1', 'y', source=stream_source)#, legend = "m/n = 0.45")
right.xaxis.axis_label = 'Chi (X)'
right.yaxis.axis_label = 'Elevation (m)'


#Set ranges for DEM image
dw = max(data['x'])- min(data['x'])
dh = max(data['y'])- min(data['y'])
xrmax = max(data['x'])
xrmin = min(data['x'])
yrmax = max(data['y'])
yrmin = min(data['y'])

dem = M_DEM[::-1] #set north-up on DEM, fix a small bug?

#Create Color palette
pal = viridis(256)
pal[0] = '#000000'

topo = figure(plot_height=700, plot_width=700, x_axis_type=None, y_axis_type=None, x_range=[xrmin,xrmax], y_range=[yrmin,yrmax])

topo.image(image=[dem], x=[min(data['x'])], y=[min(data['y'])],dw=[dw], dh=[dh], palette=pal, name='image')
topo.circle(data['x'], data['y'], color='red', size=3)


# put the subplots in a gridplot
p = layout([[left, right], [topo]])



show(p)


In [171]:
#Try to polygonize shapefile

# import rasterio
# from rasterio.features import shapes
# from osgeo import gdal, ogr, osr
# import sys


# Test_data = ("/Users/bmelosh/VagrantBoxes/LSDTopoTools/Topographic_projects/Test_data/Mandakini.bil")
# Test = gdal.Open(Test_data)
# Test_band = Test.GetRasterBand(1)
# out = "export.shp"





In [161]:
#Import dependencies and data for interactive map plot

import pandas as pd
import geopandas as gpd
from bokeh.models import HoverTool
from bokeh.palettes import RdYlBu11 as palette
from bokeh.models import LogColorMapper
import pysal as ps




grid_fp = '/Users/bmelosh/projects/LSDTopoTools/MandakiniSHP.shp'
grid = gpd.read_file(grid_fp)

In [162]:


#Dealing with data projection
# CRS = grid.crs
# print(CRS)
#convert data to a specific projection
#points['geometry'] = points['geometry'].to_crs(crs=CRS)


def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )
    
grid['x'] = grid.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
grid['y'] = grid.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)



## Define classificiations for plotting
mini = min(grid['DN'])
maxi = max(grid['DN'])
breaks = [x for x in range(mini, maxi, 50)]

# Initialize the classifier and apply it
classifier = ps.User_Defined.make(bins=breaks)
elev_classif = grid[['DN']].apply(classifier)

# Rename the classified column
elev_classif.columns = ['DN_Class']

# Join it back to the grid layer
grid = grid.join(elev_classif)


# Make a copy, drop the geometry column and create ColumnDataSource
g_df = grid.drop('geometry', axis=1).copy()
gsource = ColumnDataSource(g_df)

# Create the color mapper
color_mapper = LogColorMapper(palette=palette)


# Initialize our figure
p = figure(title="Elevations")

#setting interactive plot
my_hover = HoverTool()
my_hover.tooltips = [('Elevation (m)', '@DN')]
p.add_tools(my_hover)

# Plot grid
p.patches('x', 'y', source=gsource,
         fill_color={'field': 'DN_Class', 'transform': color_mapper},
         fill_alpha=1.0, line_color="black", line_width=0.05)



show(p)


In [98]:
import pandas as pd

data = pd.read_csv((path+name))

df = pd.DataFrame(data)

unique_channels = df.channel_number.unique() #display different channels
print(len(unique_channels))

a = df.loc[df['channel_number'] == 0]
b = df.loc[df['channel_number'] == 1]
c = df.loc[df['channel_number'] == 2]
d = df.loc[df['channel_number'] == 3]
e = df.loc[df['channel_number'] == 4]

grouped = df.groupby('channel_number')

#for loop for dividing up the channel data
# b = []
# for i in range(0, len(unique_channels)): 
#     a = df.loc[df['channel_number'] == i]
#     a.
grouped['elevation'].mean()


# for i, group in grouped:
#     #plt.figure()
#     group.plot(x='x', y='y', title=str(i))


5


channel_number
0    1635.430145
1    1515.539411
2    1248.108796
3    1180.970647
4    1340.490866
Name: elevation, dtype: float64

In [105]:
#for i in range(0, len(grouped)):
x = a['x']
y = a['y']
x2 = b['x']
y2 = b['y']

from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Circle, HoverTool, CustomJS

output_file("hover_callback.html")

# Basic plot setup
p = figure(width=600, height=300, x_axis_type="datetime", tools="", toolbar_location=None, title='Hover over points')
p.line(x, y, line_dash="4 4", line_width=1, color='gray')
p.line(x2, y2, line_dash="4 4", line_width=1, color='gray')

# Add a circle, that is visible only when selected
source = ColumnDataSource({'x': x, 'y': y})
invisible_circle = Circle(x='x', y='y', fill_color='gray', fill_alpha=0.05, line_color=None, size=20)
visible_circle = Circle(x='x', y='y', fill_color='firebrick', fill_alpha=0.5, line_color=None, size=20)
cr = p.add_glyph(source, invisible_circle, selection_glyph=visible_circle, nonselection_glyph=invisible_circle)

# Add a hover tool, that selects the circle
code = "source.set('selected', cb_data['index']);"
callback = CustomJS(args={'source': source}, code=code)
p.add_tools(HoverTool(tooltips=None, callback=callback, renderers=[cr], mode='hline'))

show(p)

In [107]:

import holoviews as hv
#import geoviews as gv
import param, paramnb
import pandas as pd

from colorcet import cm
from bokeh.models import WMTSTileSource
from holoviews.operation.datashader import datashade
from holoviews.streams import RangeXY

hv.notebook_extension('bokeh')

%time df = pd.read_csv('data/nyc_taxi.csv', usecols = ['passenger_count', \
                       'pickup_x', 'pickup_y', 'dropoff_x', 'dropoff_y'])
df.tail()

ImportError: No module named paramnb